In [1]:
# Import required libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import statsmodels.api as sm
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from mlxtend.feature_selection import SequentialFeatureSelector as SFS
from sklearn.linear_model import LinearRegression

# Load the new dataset
# Make sure "D600 Task 1 Dataset 1.csv" is in the same folder
df = pd.read_csv('D600 Task 2 Dataset 1 Housing Information.csv')

# Drop the ID column (not useful for prediction)
df = df.drop(columns=['ID'])

# Preview the data
print("Data Shape:", df.shape)
df.head()

Data Shape: (7000, 21)


,Price,SquareFootage,NumBathrooms,NumBedrooms,BackyardSpace,CrimeRate,SchoolRating,AgeOfHome,DistanceToCityCenter,EmploymentRate,...,RenovationQuality,LocalAmenities,TransportAccess,Fireplace,HouseColor,Garage,Floors,Windows,PreviousSalePrice,IsLuxury
0,255614.8992,566.62,1.000000,4,779.42,20.56,5.62,39.46,10.08,97.29,...,4.93,4.44,4.55,Yes,Blue,No,1,13,181861.54230,0
1,155586.0947,1472.34,1.000000,2,656.13,15.62,5.63,40.51,7.89,93.22,...,4.08,5.56,6.83,No,Green,No,1,17,50042.59757,0
2,131050.8324,550.00,1.779354,3,754.57,12.47,9.20,48.38,23.74,96.60,...,4.26,8.07,8.48,Yes,Green,Yes,2,34,48400.34440,0
3,151361.7125,941.81,2.035254,2,439.59,22.22,7.08,94.67,5.22,91.45,...,4.45,5.00,6.27,Yes,Red,No,1,14,84594.12145,0
4,113167.6128,550.00,1.064644,3,353.03,8.28,5.93,16.80,43.13,86.50,...,3.36,5.46,6.99,No,White,Yes,1,21,22934.59654,0
